## 先生より指示の条件

In [1]:
#df = df[df['state'].isin(['failed','successful'])]
#df["state"] = df["state"]=="successful"
#X = df[***].values
#y = df["state"].values
#test_size = 0.2
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
#random_state=1234)

## 方針：
- 変数は極力削除しない。むしろ2次変数や、変数同士の割り算等なるべく増やす方向。
- 計算が軽いLogistic Regressionで味見。
- 最終的に、勾配ブースティング学習器（xgboostを使用）が何とかしてくれることを期待。
- 可視化による検証等は行えませんでした。読みにくいですが、ご容赦ください。

## 結果(Accuracy)
- ロジスティック回帰　テスト：68.382%　　訓練：68.450%
- xgboost　　　　　　 テスト：70.907%　　訓練：73.160%

## 1. ライブラリのインポート

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection 
import sklearn.svm
import sklearn.preprocessing
import datetime
import csv
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from datetime import datetime as dt
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix 

C:\Users\macta\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\macta\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## 2. データの読み込み
### 欠損値データ、不要データの削除

In [3]:
df = pd.read_csv("ks-projects-201801.csv", parse_dates=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


### 'state'=='failed' or 'succellful'以外は削除する。'succellful'=1, 'failed'=0とする

In [5]:
df=df[(df.state == 'failed') | (df.state == 'successful')]
df.state.value_counts()

failed        197719
successful    133956
Name: state, dtype: int64

In [6]:
df.loc[df.state=='successful',["state"]]=1
df.loc[df.state=='failed',["state"]]=0

### 未来の変数は削除

In [7]:
def pre_drop_colomn(df):
    df.drop('pledged', axis=1, inplace=True)
    df.drop('backers', axis=1, inplace=True)
    df.drop('usd pledged', axis=1, inplace=True)
    df.drop('usd_pledged_real', axis=1, inplace=True)
    df.drop('goal', axis=1, inplace=True)
    return df

In [8]:
#headで確認
df = pre_drop_colomn(df)
display(df.head())

,ID,name,category,main_category,currency,deadline,launched,state,country,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,0,GB,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,0,US,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,0,US,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,0,US,5000.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,2016-02-26 13:38:27,1,US,50000.00


### 各変数のタイプ

In [9]:
df.dtypes

ID                 int64
name              object
category          object
main_category     object
currency          object
deadline          object
launched          object
state              int64
country           object
usd_goal_real    float64
dtype: object

### 'name'に欠損値あり、欠損値は' 'とする

In [10]:
df.isnull().sum()

ID               0
name             3
category         0
main_category    0
currency         0
deadline         0
launched         0
state            0
country          0
usd_goal_real    0
dtype: int64

In [11]:
df[df['name'].isnull()]

,ID,name,category,main_category,currency,deadline,launched,state,country,usd_goal_real
166851,1848699072,NaN,Narrative Film,Film & Video,USD,2012-02-29,2012-01-01 12:35:31,0,US,200000.00
307234,634871725,NaN,Video Games,Games,GBP,2013-01-06,2012-12-19 23:57:48,0,GB,3224.97
338931,796533179,NaN,Painting,Art,USD,2011-12-05,2011-11-06 23:55:55,0,US,35000.00


In [12]:
df['name'] = df['name'].fillna(' ')
df.isnull().sum()

ID               0
name             0
category         0
main_category    0
currency         0
deadline         0
launched         0
state            0
country          0
usd_goal_real    0
dtype: int64

In [13]:
df[df['ID']==1848699072]

,ID,name,category,main_category,currency,deadline,launched,state,country,usd_goal_real
166851,1848699072,,Narrative Film,Film & Video,USD,2012-02-29,2012-01-01 12:35:31,0,US,200000.0


## 3. 前処理

## 3-1 国及び通貨の検討

### 'country'=='N,0"'は'NO'ノルウェーの誤り?->単なる欠損値なのか？
https://www.wipo.int/export/sites/www/pct/guide/ja/gdvol1/annexes/annexk/ax_k.pdf

In [14]:
df['country'].unique()

array(['GB', 'US', 'CA', 'AU', 'NO', 'IT', 'DE', 'IE', 'MX', 'ES', 'SE',
       'FR', 'NZ', 'CH', 'AT', 'BE', 'DK', 'HK', 'NL', 'LU', 'SG', 'N,0"',
       'JP'], dtype=object)

In [15]:
df[df['country']=='N,0"']['country'].count()

210

In [16]:
df[df['country']=='NO']['country'].count()

582

### わからないので、'country'=='N,0"'はそのままとする

In [17]:
#df = df.replace({'country': {'N,0"': 'NO'}})

In [18]:
#df['country'].unique()

In [19]:
#df[df['country']=='NO']['country'].count()

In [20]:
df['currency'].unique()

array(['GBP', 'USD', 'CAD', 'AUD', 'NOK', 'EUR', 'MXN', 'SEK', 'NZD',
       'CHF', 'DKK', 'HKD', 'SGD', 'JPY'], dtype=object)

### 自国通貨でないかどうかのflag

In [21]:
dic_currency={'GB':'GBP','US':'USD','CA':'CAD','AU':'AUD','NO':'NOK','IT':'EUR','DE':'EUR','IE':'EUR','MX':'MXN','ES':'EUR','SE':'SEK','FR':'EUR','NZ':'NZD','CH':'CHF','AT':'EUR','BE':'EUR','DK':'DKK','HK':'HKD','NL':'EUR','LU':'EUR','SG':'SGD','JP':'JPY'}

In [22]:
df['own_currency'] = df['country'].map(dic_currency)
df['use_own_currency'] = df['currency']==df['own_currency']

### EUROを使っているかのflag

In [23]:
list_EUR = ['IT', 'DE', 'IE', 'ES', 'FR', 'AT', 'BE', 'NL', 'LU']

In [24]:
df.loc[df.use_own_currency==True,['use_own_currency']]=1
df.loc[df.use_own_currency==False,['use_own_currency']]=0

In [25]:
df['use_EUR']=df['country'].isin(list_EUR)
df.loc[df.use_EUR==True,['use_EUR']]=1
df.loc[df.use_EUR==False,['use_EUR']]=0

## 3-2 'category'について
### 'category' == 'main_category'であるかどうかのflag

In [26]:
list_main_category = df['main_category'].unique()
print(list_main_category)

['Publishing' 'Film & Video' 'Music' 'Food' 'Crafts' 'Games' 'Design'
 'Comics' 'Fashion' 'Theater' 'Art' 'Photography' 'Technology' 'Dance'
 'Journalism']


In [27]:
df['no_sub_div_category']= (df['main_category']==df['category'])
df.loc[df.no_sub_div_category==True,['no_sub_div_category']]=1
df.loc[df.no_sub_div_category==False,['no_sub_div_category']]=0

## 3-3 'name'についての検討

### 頻出単語を含むかどうかを説明変数に追加（1000回以上出てくるものを目安）
### 'name'を全部小文字化

In [28]:
df['name']=df['name'].str.lower()

### 計算が少し重いので、一旦csvに保存

In [29]:
def count_word(df):
    word_dic = {}
    for tmp in list(df['ID']):
        list_name_split = df[df['ID']==tmp]['name'].str.split().values.tolist()
        temp_word=list_name_split[0]
        if type(temp_word)!= float:
            for temp_word in list_name_split[0]:
                if str(temp_word) in word_dic:
                    word_dic[str(temp_word.lower())] =  word_dic[str(temp_word).lower()]+1
                else:
                    word_dic.setdefault(str(temp_word.lower()), 1)
    return word_dic

In [30]:
#count_word(df)
#word_dic_sort = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)

In [31]:
#df_word_dic = pd.DataFrame(word_dic_sort)
#df_word_dic.to_csv('word_dic_sort.csv')

### 'the','a'などの接頭語等は手動で削除

### 抽出した単語を表示

In [32]:
ws= pd.read_csv("word_dic_read.csv",parse_dates=True)
ws.values.squeeze()

array(['album', 'film', 'project', 'book', 'art', 'game', 'music',
       'first', 'help', 'short', 'debut', 'documentary', 'series', 'life',
       'ep', 'world', 'video', 'love', 'make', 'story', 'novel', 'tour',
       'app', 'all', 'food', 'cd', ':', 'show', 'me', 'card', 'studio',
       'get', 'festival', 'record', 'feature', 'made', '\\/',
       'collection', 'movie', 'time', 'magazine', 'clothing', 'dance',
       'playing', 'recording', 'comic', 'fund', 'american', 'free',
       'home', 'live', 'adventure', 'city', 'mobile', 'little', 'kids',
       'making', '3d', 'edition', '\\+', 'release', 'man', 'children',
       'full', 'black', 'journey', 'big', 'design', 'photography',
       'company', 'coffee', 'fashion', 'dream', 'fantasy', 'best', 'day',
       'light', 'community', 'space', 'way', 'play', 'more', 'board',
       'bring', 'graphic', 'horror', 'season', 'house', 'custom', 'out',
       'musical', 'back', 'rpg', 'original', 'comedy', 'system', 'not',
       'appar

In [33]:
for j in ws.values.squeeze():
    df['contain_'+j]=df['name'].str.contains(j)*1

### 'name'の長さを説明変数に追加

In [34]:
#nameの長さ
df['name_len'] = (df['name'].str).len()

### 'name'の単語数≒スペースの数を説明変数に追加

In [35]:
#単語数
df["name_words"] = df.name.str.count(" ")

### 不採用　IDの桁数

In [36]:
#df['ID_digit']=np.log10(df['ID'])

## 3-4 日付及び曜日について

### 締切日までの日数を説明変数に追加

In [37]:
df['launched'] = pd.to_datetime(df['launched'], format='%Y-%m-%d %H:%M:%S')
df['deadline'] = pd.to_datetime(df['deadline'], format='%Y-%m-%d %H:%M:%S')
df['duration'] = (df['deadline'] - df['launched']).dt.days

### 同日が締切日もある
#### df[df['duration']==0]　は、小数で表現

In [38]:
df.loc[df.duration==0,['duration']]=(df['launched'].dt.hour*3600+df['launched'].dt.minute*60+df['launched'].dt.second)/(24*60*60)

### 曜日を説明変数に追加

In [39]:
df["launched_dayofweek"] = df['launched'].dt.dayofweek
df["deadline_dayofweek"] = df['deadline'].dt.dayofweek

### 目標金額÷締切期間を説明変数に追加

In [40]:
df['goal_per_duration']=df['usd_goal_real']/df['duration']

### 年について説明変数に追加

In [41]:
print(df['launched'].dt.year.min())
print(df['deadline'].dt.year.max())

2009
2018


In [42]:
df['dead_line_year']= (df['deadline'].dt.year-2013.5)/4.5
#df['dead_line_year'].describe()

### 締切年の2次を説明変数に追加(景気の変動など)

In [43]:
df['dead_line_year_2nd']= ((df['deadline'].dt.year-2013.5)/4.5)**2
#df['dead_line_year_2nd'].describe()

In [44]:
df['launched_year']= (df['deadline'].dt.year-2013.5)/4.5
#df['launched_year'].describe()

### 開始年の2次を説明変数に追加(景気の変動など)

In [45]:
df['launched_year_2nd']= ((df['deadline'].dt.year-2013.5)/4.5)**2
#df['launched_year_2nd'].describe()

### 季節変動をsin, cosで表現し、説明変数に追加

In [46]:
def calc_season_sin_cos(df,lord):
    df['tmp_season'+lord]=df[lord].dt.year.astype(str)
    start_date_list = ['-01-01']*df['ID'].count()
    start_date_ser = pd.Series(start_date_list, index=df['tmp_season'+lord].index)
    df['DayofYear_lord']=df['tmp_season'+lord] + start_date_ser
    df['DayofYear_lord']=pd.to_datetime(df['DayofYear_lord'], format='%Y-%m-%d %H:%M:%S')
    df['DayofYear_lord']=(df[lord]-df['DayofYear_lord']).dt.days
    tmp_days = np.array((df['DayofYear_lord']))
    df.drop(columns='DayofYear_lord', axis=1, inplace=True)
    df.drop(columns='tmp_season'+lord, axis=1, inplace =True)
    ss = np.sin(np.radians(tmp_days/365*360))
    cc = np.cos(np.radians(tmp_days/365*360))
    return pd.Series(ss, index=df['ID'].index, name=str(lord)+'_sin'),pd.Series(cc, index=df['ID'].index,name=str(lord)+'_cos')

In [47]:
launched_sin, launched_cos =  calc_season_sin_cos(df,'launched')
df = pd.concat([df, launched_sin, launched_cos], axis=1)

deadline_sin, deadline_cos =  calc_season_sin_cos(df,'deadline')
df = pd.concat([df, deadline_sin, deadline_cos], axis=1)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331675 entries, 0 to 378660
Columns: 149 entries, ID to deadline_cos
dtypes: datetime64[ns](2), float64(11), int32(121), int64(9), object(6)
memory usage: 236.5+ MB


In [49]:
df.describe()

,ID,state,usd_goal_real,use_own_currency,use_EUR,no_sub_div_category,contain_album,contain_film,contain_project,contain_book,...,deadline_dayofweek,goal_per_duration,dead_line_year,dead_line_year_2nd,launched_year,launched_year_2nd,launched_sin,launched_cos,deadline_sin,deadline_cos
count,3.316750e+05,331675.000000,3.316750e+05,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,...,331675.000000,3.316750e+05,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000
mean,1.074271e+09,0.403877,4.151000e+04,0.999367,0.043286,0.265522,0.038918,0.033488,0.032665,0.036916,...,3.183773,2.384553e+03,0.165449,0.209278,0.165449,0.209278,0.009507,-0.052123,-0.019778,-0.052338
std,6.191924e+08,0.490674,1.108930e+06,0.025155,0.203501,0.441611,0.193399,0.179906,0.177757,0.188555,...,1.951072,4.024763e+05,0.426503,0.210910,0.426503,0.210910,0.719422,0.692552,0.714654,0.697239
min,5.971000e+03,0.000000,1.000000e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.250000e-03,-1.000000,0.012346,-1.000000,0.012346,-0.999991,-0.999963,-0.999991,-0.999963
25%,5.371725e+08,0.000000,2.000000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,6.666667e+01,-0.111111,0.012346,-0.111111,0.012346,-0.723644,-0.744104,-0.723644,-0.744104
50%,1.074573e+09,0.000000,5.000000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,1.724138e+02,0.111111,0.111111,0.111111,0.111111,0.034422,-0.064508,-0.068802,-0.098820
75%,1.609844e+09,1.000000,1.500000e+04,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,5.000000e+02,0.555556,0.308642,0.555556,0.308642,0.735417,0.625411,0.717677,0.625411
max,2.147476e+09,1.000000,1.663614e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,6.000000,1.693122e+08,1.000000,1.000000,1.000000,1.000000,0.999991,1.000000,0.999991,1.000000


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331675 entries, 0 to 378660
Columns: 149 entries, ID to deadline_cos
dtypes: datetime64[ns](2), float64(11), int32(121), int64(9), object(6)
memory usage: 236.5+ MB


In [51]:
df.head()

,ID,name,category,main_category,currency,deadline,launched,state,country,usd_goal_real,...,deadline_dayofweek,goal_per_duration,dead_line_year,dead_line_year_2nd,launched_year,launched_year_2nd,launched_sin,launched_cos,deadline_sin,deadline_cos
0,1000002330,the songs of adelaide & abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,0,GB,1533.95,...,4,26.447414,0.333333,0.111111,0.333333,0.111111,-0.628763,-0.777597,-0.992222,0.124479
1,1000003930,greeting from earth: zgac arts capsule for et,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,0,US,30000.00,...,2,508.474576,0.777778,0.604938,0.777778,0.604938,-0.871706,-0.490029,-0.867456,0.497513
2,1000004038,where is hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,0,US,45000.00,...,1,1022.727273,-0.111111,0.012346,-0.111111,0.012346,0.188227,0.982126,0.821477,0.570242
3,1000007540,toshicapital rekordz needs help to complete album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,0,US,5000.00,...,0,172.413793,-0.333333,0.111111,-0.333333,0.111111,0.965740,0.259512,0.967938,-0.251190
5,1000014025,monarch espresso bar,Restaurants,Food,USD,2016-04-01,2016-02-26 13:38:27,1,US,50000.00,...,4,1470.588235,0.555556,0.308642,0.555556,0.308642,0.821477,0.570242,0.999991,0.004304


## 4. 学習用データに整理

### 使わない変数(object等)を削除

In [52]:
df.drop(columns='ID', axis=1, inplace=True)
df.drop(columns='name', axis=1, inplace=True)
df.drop(columns='currency', axis=1, inplace=True)
df.drop(columns='deadline', axis=1, inplace=True)
df.drop(columns='launched', axis=1, inplace=True)
df.head()

,category,main_category,state,country,usd_goal_real,own_currency,use_own_currency,use_EUR,no_sub_div_category,contain_album,...,deadline_dayofweek,goal_per_duration,dead_line_year,dead_line_year_2nd,launched_year,launched_year_2nd,launched_sin,launched_cos,deadline_sin,deadline_cos
0,Poetry,Publishing,0,GB,1533.95,GBP,1,0,0,0,...,4,26.447414,0.333333,0.111111,0.333333,0.111111,-0.628763,-0.777597,-0.992222,0.124479
1,Narrative Film,Film & Video,0,US,30000.00,USD,1,0,0,0,...,2,508.474576,0.777778,0.604938,0.777778,0.604938,-0.871706,-0.490029,-0.867456,0.497513
2,Narrative Film,Film & Video,0,US,45000.00,USD,1,0,0,0,...,1,1022.727273,-0.111111,0.012346,-0.111111,0.012346,0.188227,0.982126,0.821477,0.570242
3,Music,Music,0,US,5000.00,USD,1,0,1,1,...,0,172.413793,-0.333333,0.111111,-0.333333,0.111111,0.965740,0.259512,0.967938,-0.251190
5,Restaurants,Food,1,US,50000.00,USD,1,0,0,0,...,4,1470.588235,0.555556,0.308642,0.555556,0.308642,0.821477,0.570242,0.999991,0.004304


### one-hot encoding

In [53]:
df = pd.get_dummies(df)

In [54]:
df.head()

,state,usd_goal_real,use_own_currency,use_EUR,no_sub_div_category,contain_album,contain_film,contain_project,contain_book,contain_art,...,own_currency_EUR,own_currency_GBP,own_currency_HKD,own_currency_JPY,own_currency_MXN,own_currency_NOK,own_currency_NZD,own_currency_SEK,own_currency_SGD,own_currency_USD
0,0,1533.95,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,30000.00,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0,45000.00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,5000.00,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,1,50000.00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [55]:
df.describe()

,state,usd_goal_real,use_own_currency,use_EUR,no_sub_div_category,contain_album,contain_film,contain_project,contain_book,contain_art,...,own_currency_EUR,own_currency_GBP,own_currency_HKD,own_currency_JPY,own_currency_MXN,own_currency_NOK,own_currency_NZD,own_currency_SEK,own_currency_SGD,own_currency_USD
count,331675.000000,3.316750e+05,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,...,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000
mean,0.403877,4.151000e+04,0.999367,0.043286,0.265522,0.038918,0.033488,0.032665,0.036916,0.069743,...,0.043286,0.088804,0.001438,0.000069,0.004254,0.001755,0.003841,0.004550,0.001369,0.788000
std,0.490674,1.108930e+06,0.025155,0.203501,0.441611,0.193399,0.179906,0.177757,0.188555,0.254714,...,0.203501,0.284461,0.037896,0.008327,0.065085,0.041853,0.061858,0.067297,0.036972,0.408725
min,0.000000,1.000000e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,5.000000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.500000e+04,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.663614e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [56]:
df.isnull().sum()

state                  0
usd_goal_real          0
use_own_currency       0
use_EUR                0
no_sub_div_category    0
contain_album          0
contain_film           0
contain_project        0
contain_book           0
contain_art            0
contain_game           0
contain_music          0
contain_first          0
contain_help           0
contain_short          0
contain_debut          0
contain_documentary    0
contain_series         0
contain_life           0
contain_ep             0
contain_world          0
contain_video          0
contain_love           0
contain_make           0
contain_story          0
contain_novel          0
contain_tour           0
contain_app            0
contain_all            0
contain_food           0
                      ..
country_ES             0
country_FR             0
country_GB             0
country_HK             0
country_IE             0
country_IT             0
country_JP             0
country_LU             0
country_MX             0


In [57]:
X = np.array(df.drop('state', axis=1).values)
y = df["state"].values

In [58]:
X.shape

(331675, 350)

### ※指示条件

In [59]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=1234)

### 標準化

In [60]:
scaler=StandardScaler()
scaler.fit(X_train)

X_train_sc=scaler.transform(X_train)
X_test_sc=scaler.transform(X_test)

## 5. 学習／評価

## 5-1 ロジスティック回帰で試行

In [61]:
lr = SGDClassifier(loss='log', penalty='none', max_iter=1000, tol=1e-3, random_state=1234)
lr.fit(X_train_sc, y_train)
# 予測
y_pred_train_lr = lr.predict(X_train_sc)
y_pred_test_lr = lr.predict(X_test_sc)

# 正答率
acc_score_train_lr = accuracy_score(y_train,y_pred_train_lr)
acc_score_test_lr = accuracy_score(y_test, y_pred_test_lr)

print('正答率（Accuracy_train_logistic） = {:.3f}%'.format(100 * acc_score_train_lr ))
print('正答率（Accuracy_test_logistic） = {:.3f}%'.format(100 * acc_score_test_lr))

正答率（Accuracy_train_logistic） = 68.450%
正答率（Accuracy_test_logistic） = 68.382%


## 5-2 XGboostを実行

In [62]:
def to_zero_one_class(pred):
    result = np.round(pred)
    result[result < 0.5] = 0
    result[result >= 0.5] = 1
    return result

### パラメータはあまり調整できず。seedで平均化を試行。

In [63]:
xgbcs = []
for i in range(5):
    xgbc = xgb.XGBClassifier(#class_weight="balanced",
                             learning_rate=0.1,
                             max_depth=6,
                             min_child_weight=2,
                             gamma = 0.0,
                             n_estimators=400,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             reg_lambda=1.0,
                             scale_pos_weight=1,
                             eval_metric="logloss",
                             seed=i)
    print(i)
    xgbc.fit(X_train_sc, y_train)
    xgbcs.append(xgbc)
y_predict_train_xgb = np.array([xgbc.predict_proba(X_train_sc)[:, 1]for xgbc in xgbcs]).mean(axis=0)
y_predict_test_xgb = np.array([xgbc.predict_proba(X_test_sc)[:, 1]for xgbc in xgbcs]).mean(axis=0)

0
1
2
3
4


In [64]:
acc_score_train_xgb = accuracy_score(y_train,  to_zero_one_class(y_predict_train_xgb))
acc_score_test_xgb = accuracy_score(y_test,  to_zero_one_class(y_predict_test_xgb))
print('正答率（Accuracy_train_xgb） = {:.3f}%'.format(100 * acc_score_train_xgb))
print('正答率（Accuracy_test_xgb） = {:.3f}%'.format(100 * acc_score_test_xgb))

正答率（Accuracy_train_xgb） = 73.160%
正答率（Accuracy_test_xgb） = 70.907%
